# Fine-tuning a Pretrained Model

Learning how to fine-tune a pretrained model with a classification head

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 20/11/2025   | Martin | Created   | Created to learn how to fine-tune a pretrained model | 

# Content

* [Introduction](#introduction)

# Introduction

Training BERT with MRPC dataset that indicates whether a _pair of sentences are paraphrased or not_

In [1]:
%load_ext watermark

In [2]:
import torch
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification

Sample training loop using 2 sentences

In [3]:
# Define the original model - text classification model
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

# Additional sequences to train on
sequences = [
  "I've been waiting for a HuggingFace course my whole life.",
  "This course is amazing!",
]

# Process the sequences
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
batch["labels"] = torch.tensor([1, 1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


---

# Training Pipeline

The original BERT model was used to predict whether the second sentence follows the first. Sentences in the dataset were split and half of them were randomly paired with another half from another sentence to form the negative examples.

`token_type_id` represents whether a sentence belongs to the first or second half. But this can be ignored given that we are fine-tuning on a different objective. As long as the model remains the same, the targets can be changed

<u>Additional Components</u>

- `datasets` is a Huggingface API for loading datasets to and from their Hub

💡<u>Tips</u>

- Padding batches is more efficient since it only pads to the maximum length of the batch
  - `DataCollatorWithPadding` does this by passing the relevant tokenizer as input

In [36]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

In [12]:
CHECKPOINT = "bert-base-uncased"

In [5]:
raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

README.md: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [6]:
raw_train_dataset = raw_datasets['train']
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [7]:
raw_train_dataset.features

{'sentence1': Value('string'),
 'sentence2': Value('string'),
 'label': ClassLabel(names=['not_equivalent', 'equivalent']),
 'idx': Value('int32')}

Preprocessing the data - tokenization

In [18]:
tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)
sent_1 = tokenizer(raw_datasets['train']['sentence1'][0])
sent_2 = tokenizer(raw_datasets['train']['sentence2'][0])

print("----- Comparing Tokenized Sentences ------")
print(sent_1)
print(sent_2)

----- Comparing Tokenized Sentences ------
{'input_ids': [101, 2572, 3217, 5831, 5496, 2010, 2567, 1010, 3183, 2002, 2170, 1000, 1996, 7409, 1000, 1010, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'input_ids': [101, 7727, 2000, 2032, 2004, 2069, 1000, 1996, 7409, 1000, 1010, 2572, 3217, 5831, 5496, 2010, 2567, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
inputs = tokenizer(
  "This is the first sentence.",
  "This is the second one.",
)
print(inputs)

decoded = tokenizer.convert_ids_to_tokens(inputs['input_ids'])
print(decoded)

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['[CLS]', 'this', 'is', 'the', 'first', 'sentence', '.', '[SEP]', 'this', 'is', 'the', 'second', 'one', '.', '[SEP]']


In [28]:
# .map() method applies a function on each element of the dataset
def tokenize_function(val):
  return tokenizer(val['sentence1'], val['sentence2'], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

Using the data collator to pad by batches

In [39]:
# Define the collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

samples = tokenized_datasets['train'][:8] # Take 8 samples
samples = {k: v for k, v in samples.items() if k not in ['idx', 'sentence1', 'sentence2']}

print("Variable sequence length for each sentence in the tokenized dataset")
print([len(x) for x in samples['input_ids']])
print()

batch = data_collator(samples)
print("Dynamically padded for the samples in this batch of size 8")
print({k: v.shape for k, v in batch.items()})

Variable sequence length for each sentence in the tokenized dataset
[50, 59, 47, 67, 59, 50, 62, 32]

Dynamically padded for the samples in this batch of size 8
{'input_ids': torch.Size([8, 67]), 'token_type_ids': torch.Size([8, 67]), 'attention_mask': torch.Size([8, 67]), 'labels': torch.Size([8])}


In [ ]:
%watermark